In [ ]:
# Импорт используемых библиотек
! pip install catboost

import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.1 MB/s eta 0:00:00


In [ ]:
# Загрузка дата-сетов.

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")


In [ ]:
#Смотрим на типы всех фич тест-датафрейма

train_data.dtypes

# Признак TotalSpent - object , но тк он исчисляемый , его нужно привести к типу float. Чтобы сделать это нужно заменить в
# нем пустые строки.
train_data.TotalSpent = np.where(train_data.TotalSpent == " " , '1' , train_data.TotalSpent).astype(float)

# Заменить все значения Na на среднее по фиче, если таковые имеются .
train_data.TotalSpent.fillna( train_data.TotalSpent.mean() , inplace = True )


In [ ]:
# Формируем X_train , y_train

y_train = train_data["Churn"]
X_train = train_data.drop(['Churn'], axis=1)




In [ ]:
# Применяем алгоритм CatBoostCalssifier , ему нужно передать индексы всех категориальных признаков (строка ниже)


cat_cols = [i for i in range(3,19)]


clf = CatBoostClassifier(

    iterations = 200 , eval_metric = 'AUC:hints=skip_train~false' , custom_loss = 'AUC' , cat_features = cat_cols
)

# Будет строить деревья глубиной 200 , минимизировать ошибку на метрике ROC-AUC .

params = { 'max_depth' : [5 , 6 , 7 , 8 , 9] ,
          'learning_rate' : [0.1 , 0.2 , 0.3 , 0.4 , 0.5 ]
         }

# Подбор параметров макс глубины каждого дерева и lr для градиентного спуска.

grid = GridSearchCV( clf , params , cv = 5 , n_jobs = -1 )

grid.fit( X_train , y_train  )


 # Естиматор с параметрами , которые подобрал GridSearch



0:	learn: 0.8127291	total: 63.3ms	remaining: 12.6s
1:	learn: 0.8251149	total: 78.9ms	remaining: 7.81s
2:	learn: 0.8293019	total: 93.7ms	remaining: 6.16s
3:	learn: 0.8348190	total: 109ms	remaining: 5.32s
4:	learn: 0.8364256	total: 124ms	remaining: 4.83s
5:	learn: 0.8370364	total: 135ms	remaining: 4.38s
6:	learn: 0.8375365	total: 150ms	remaining: 4.13s
7:	learn: 0.8389441	total: 162ms	remaining: 3.89s
8:	learn: 0.8395947	total: 175ms	remaining: 3.72s
9:	learn: 0.8412010	total: 190ms	remaining: 3.6s
10:	learn: 0.8419957	total: 200ms	remaining: 3.43s
11:	learn: 0.8442517	total: 215ms	remaining: 3.37s
12:	learn: 0.8454882	total: 230ms	remaining: 3.31s
13:	learn: 0.8464207	total: 244ms	remaining: 3.24s
14:	learn: 0.8475814	total: 267ms	remaining: 3.3s
15:	learn: 0.8483199	total: 282ms	remaining: 3.24s
16:	learn: 0.8488156	total: 296ms	remaining: 3.19s
17:	learn: 0.8499090	total: 309ms	remaining: 3.13s
18:	learn: 0.8505293	total: 322ms	remaining: 3.07s
19:	learn: 0.8511779	total: 336ms	remain

In [ ]:
best_clf = grid.best_estimator_

best_clf.fit(X_train , y_train)
X_test = test_data
y_pred = best_clf.predict_proba(X_test )[ : , 1 ] # Вероятности отнесения к классам 0/1 .
y_pred

0:	learn: 0.8127291	total: 23.1ms	remaining: 4.6s
1:	learn: 0.8251149	total: 40.9ms	remaining: 4.05s
2:	learn: 0.8293019	total: 54.6ms	remaining: 3.58s
3:	learn: 0.8348190	total: 68.3ms	remaining: 3.35s
4:	learn: 0.8364256	total: 83.6ms	remaining: 3.26s
5:	learn: 0.8370364	total: 95.1ms	remaining: 3.08s
6:	learn: 0.8375365	total: 108ms	remaining: 2.98s
7:	learn: 0.8389441	total: 120ms	remaining: 2.88s
8:	learn: 0.8395947	total: 133ms	remaining: 2.83s
9:	learn: 0.8412010	total: 148ms	remaining: 2.81s
10:	learn: 0.8419957	total: 159ms	remaining: 2.72s
11:	learn: 0.8442517	total: 173ms	remaining: 2.71s
12:	learn: 0.8454882	total: 187ms	remaining: 2.69s
13:	learn: 0.8464207	total: 201ms	remaining: 2.67s
14:	learn: 0.8475814	total: 216ms	remaining: 2.66s
15:	learn: 0.8483199	total: 234ms	remaining: 2.69s
16:	learn: 0.8488156	total: 249ms	remaining: 2.69s
17:	learn: 0.8499090	total: 263ms	remaining: 2.66s
18:	learn: 0.8505293	total: 276ms	remaining: 2.63s
19:	learn: 0.8511779	total: 299ms	re

array([0.03870605, 0.70860383, 0.7146863 , ..., 0.85946374, 0.57666218,
       0.00751745])

In [ ]:
# Можно было проще через df.index , но я сгенерировал отдельно колонку индексов .

submission = pd.DataFrame({'Id' : [i for i in range(1761)] , 'Churn' : y_pred})

In [ ]:
submission

,Id,Churn
0,0,0.038706
1,1,0.708604
2,2,0.714686
3,3,0.015449
4,4,0.003080
...,...,...
1756,1756,0.118256
1757,1757,0.008646
1758,1758,0.859464
1759,1759,0.576662


In [ ]:
# Сохраняем файл в директорию юпитера
submission.to_csv('submission_kaggle.csv' , index = False)